In [4]:
import geopandas as gpd

# Load the GeoJSON files
zip_codes = gpd.read_file("../data/raw/zipcodes/ZipCodes_3170383398510875265/California_Zip_Codes.shp")
climate_zones = gpd.read_file("../data/raw/climate-zones/Climate_Zones_-_DOE_Building_America_Program.geojson")

# Ensure both GeoDataFrames use the same CRS
zip_codes = zip_codes.to_crs(climate_zones.crs)

# Perform a spatial join to map ZIP codes to climate zones
# This assumes that the geometry column is named 'geometry' in both GeoDataFrames
joined = gpd.sjoin(zip_codes, climate_zones, how="left", op="within")

# The resulting DataFrame will have a column for ZIP codes and their corresponding climate zones
# Save the result to a new GeoJSON file
joined.to_file("zip_codes_with_climate_zones.geojson", driver="GeoJSON")

# Load the resulting GeoJSON file with ZIP codes and climate zones
joined = gpd.read_file("zip_codes_with_climate_zones.geojson")



/Users/ana/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
print("First few rows of the data:")
print(joined.head())


# Extract ZIP codes and their corresponding climate zones into a dictionary
zip_to_climate_zone = {}
for idx, row in joined.iterrows():
    zip_code = row['ZIP_CODE']  # Replace with the actual column name for ZIP codes
    climate_zone = row['BA_Climate_Zone']  # Replace with the actual column name for climate zones
    zip_to_climate_zone[zip_code] = climate_zone

# Print the dictionary or save it to a file
print(zip_to_climate_zone)

import json
with open('zip_to_climate_zone.json', 'w') as f:
    json.dump(zip_to_climate_zone, f)

First few rows of the data:
  ZIP_CODE                  PO_NAME STATE  POPULATION  POP_SQMI    SQMI  \
0    00012          Mt Meadows Area    CA         -99     -99.0   30.92   
1    00016  Sequoia National Forest    CA         -99     -99.0   39.33   
2    00017  Northeast Fresno County    CA         -99     -99.0  564.38   
3    00018    Los Padres Ntl Forest    CA         -99     -99.0   90.83   
4    00019        Lassen Ntl Forest    CA         -99     -99.0  412.59   

   index_right  OBJECTID  IECC_Climate_Zone IECC_Moisture_Regime  \
0         13.0      14.0                5.0                    B   
1          6.0       7.0                3.0                    B   
2          NaN       NaN                NaN                 None   
3          7.0       8.0                3.0                    C   
4         13.0      14.0                5.0                    B   

  BA_Climate_Zone  Count_  AnalysisArea  \
0            Cold   150.0  1.223563e+06   
1         Hot-Dry   109.0 

In [8]:
import geopandas as gpd
from shapely.geometry import Point

# Load the GeoJSON files
zip_codes = gpd.read_file("../data/raw/zipcodes/ZipCodes_3170383398510875265/California_Zip_Codes.shp")
climate_zones = gpd.read_file("../data/raw/climate-zones/Climate_Zones_-_DOE_Building_America_Program.geojson")

# Ensure both GeoDataFrames use the same CRS
zip_codes = zip_codes.to_crs(climate_zones.crs)

# Calculate centroids of the zip code polygons
zip_codes['centroid'] = zip_codes.geometry.centroid

# Perform a spatial join to map ZIP codes to climate zones using centroids
joined = gpd.sjoin(zip_codes.set_geometry('centroid'), climate_zones, how="left", op="intersects")

# Revert to original geometry
joined = joined.set_geometry('geometry')

# For ZIP codes with null climate zones, assign the nearest climate zone
null_climate_zones = joined[joined['BA_Climate_Zone'].isnull()]
non_null_climate_zones = joined.dropna(subset=['BA_Climate_Zone'])

for idx, row in null_climate_zones.iterrows():
    centroid = row['centroid']
    closest_zone = non_null_climate_zones.distance(centroid).idxmin()
    joined.at[idx, 'BA_Climate_Zone'] = joined.at[closest_zone, 'BA_Climate_Zone']

# Drop the centroid column
joined = joined.drop(columns=['centroid'])

# Save the result to a new GeoJSON file
joined.to_file("zip_codes_with_climate_zones.geojson", driver="GeoJSON")

# Load the resulting GeoJSON file with ZIP codes and climate zones
joined = gpd.read_file("zip_codes_with_climate_zones.geojson")

print("First few rows of the data:")
print(joined.head())

# Extract ZIP codes and their corresponding climate zones into a dictionary
zip_to_climate_zone = {}
for idx, row in joined.iterrows():
    zip_code = row['ZIP_CODE']  # Replace with the actual column name for ZIP codes
    climate_zone = row['BA_Climate_Zone']  # Replace with the actual column name for climate zones
    zip_to_climate_zone[zip_code] = climate_zone

# Print the dictionary or save it to a file
print(zip_to_climate_zone)

import json
with open('zip_to_climate_zone_filled.json', 'w') as f:
    json.dump(zip_to_climate_zone, f)

/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_66542/3850932757.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zip_codes['centroid'] = zip_codes.geometry.centroid
/Users/ana/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_66542/3850932757.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_zone = non_null_climate_zones.distance(centroid).idxmin()
/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_6

First few rows of the data:
  ZIP_CODE                  PO_NAME STATE  POPULATION  POP_SQMI    SQMI  \
0    00012          Mt Meadows Area    CA         -99     -99.0   30.92   
1    00016  Sequoia National Forest    CA         -99     -99.0   39.33   
2    00017  Northeast Fresno County    CA         -99     -99.0  564.38   
3    00018    Los Padres Ntl Forest    CA         -99     -99.0   90.83   
4    00019        Lassen Ntl Forest    CA         -99     -99.0  412.59   

   index_right  OBJECTID  IECC_Climate_Zone IECC_Moisture_Regime  \
0         13.0      14.0                5.0                    B   
1          6.0       7.0                3.0                    B   
2          6.0       7.0                3.0                    B   
3          7.0       8.0                3.0                    C   
4         13.0      14.0                5.0                    B   

  BA_Climate_Zone  Count_  AnalysisArea  \
0            Cold   150.0  1.223563e+06   
1         Hot-Dry   109.0 

In [9]:
# Check for null values in the dictionary
null_values = [zip_code for zip_code, climate_zone in zip_to_climate_zone.items() if climate_zone is None]

if null_values:
    print("The following ZIP codes have null climate zones:", null_values)
else:
    print("All ZIP codes have valid climate zones.")

All ZIP codes have valid climate zones.
